# Mount Google Drive dan Konfigurasi Direktori

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted.")

# Konfigurasi Direktori
# Sesuaikan BASE_DRIVE_PATH dengan lokasi folder 'UAS Penalaran Komputer' di Google Drive Anda
BASE_DRIVE_PATH = '/content/drive/My Drive/UAS Penalaran Komputer/'

# Direktori sumber file .txt (hasil cleaning dari Tahap 1. Case Base)
RAW_DATA_DIR = os.path.join(BASE_DRIVE_PATH, 'txt_files')

# Direktori untuk menyimpan hasil representasi
PROCESSED_DATA_DIR = os.path.join(BASE_DRIVE_PATH, 'data_processed') # Nama folder baru di dalam UAS Penalaran Komputer/
OUTPUT_FILENAME_CSV = 'cases_representation.csv' # Nama file output CSV

# Buat direktori jika belum ada
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"\nDirektori file mentah (TXT): {RAW_DATA_DIR}")
print(f"Direktori output (CSV): {PROCESSED_DATA_DIR}")

Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted.

Direktori file mentah (TXT): /content/drive/My Drive/UAS Penalaran Komputer/txt_files
Direktori output (CSV): /content/drive/My Drive/UAS Penalaran Komputer/data_processed


# Import Library dan Fungsi Ekstraksi Metadata

In [ ]:
import pandas as pd
import re

def extract_metadata(text):
    """
    Ekstrak metadata: no_perkara, tanggal, jenis_perkara, pasal, pihak.
    Disesuaikan dengan format file .txt yang diberikan.
    """
    metadata = {
        'no_perkara': None,
        'tanggal': None,
        'jenis_perkara': "Pidana Khusus Narkotika & Psikotropika", # Sesuai tugas
        'pasal': None,
        'pihak': None,
    }

    # Nomor Perkara: Contoh "Nomor 713/Pid.Sus/2024/PN Rap"
    no_perkara_match = re.search(r'Nomor\s*(\d+/[A-Za-z\.]+/(\d{4})/PN\s*[A-Za-z]+)', text, re.IGNORECASE)
    if no_perkara_match:
        metadata['no_perkara'] = no_perkara_match.group(1).strip()

    # Tanggal: Contoh "tanggal 21 Mei 2024" atau di bagian akhir
    tanggal_match = re.search(r'(?:tanggal|Tgl\.)\s*(\d{1,2}\s(?:Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)\s\d{4})', text, re.IGNORECASE)
    if tanggal_match:
        metadata['tanggal'] = tanggal_match.group(1).strip()
    else: # Cari di 500 karakter terakhir
        tanggal_akhir_match = re.search(r'\s*([A-Za-z]+),\s*(\d{1,2}\s(?:Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)\s\d{4})', text[-500:], re.IGNORECASE)
        if tanggal_akhir_match:
            metadata['tanggal'] = tanggal_akhir_match.group(2).strip()

    # Pasal Digunakan: Cari pola "Pasal X" atau "melanggar Pasal Y"
    pasal_list = re.findall(r'(?:Pasal)\s*([\d\.\sA-Za-z]+)(?:\sUndang-Undang|$)', text, re.IGNORECASE)
    if pasal_list:
        clean_pasal = [p.strip() for p in pasal_list if len(p.strip()) > 2]
        metadata['pasal'] = "; ".join(sorted(list(set(clean_pasal))))
    else: # Alternatif jika konteksnya 'melanggar pasal'
        pelanggaran_pasal = re.findall(r'melanggar\s(?:ketentuan\s)?(?:Pasal|Undang-Undang)\s*([\w\d\s\-\.,]+)(?:\sUURI)?', text, re.IGNORECASE)
        if pelanggaran_pasal:
            clean_pelanggaran_pasal = [p.strip() for p in pelanggaran_pasal if len(p.strip()) > 2]
            metadata['pasal'] = "; ".join(sorted(list(set(clean_pelanggaran_pasal))))

    # Pihak: Cari "Jaksa Penuntut Umum" dan "Terdakwa"
    pihak_jpu_match = re.search(r'Jaksa Penuntut Umum\s*:\s*(.+?);', text, re.IGNORECASE)
    pihak_terdakwa_match = re.search(r'Terdakwa\s*:\s*(.+?)\n', text, re.IGNORECASE)

    pihak_str = ""
    if pihak_jpu_match:
        pihak_str += "Jaksa Penuntut Umum"
    if pihak_terdakwa_match:
        terdakwa_name = pihak_terdakwa_match.group(1).split(',')[0].strip()
        if pihak_str:
            pihak_str += f" vs. Terdakwa {terdakwa_name}"
        else:
            pihak_str += f"Terdakwa {terdakwa_name}"
    metadata['pihak'] = pihak_str if pihak_str else None

    return metadata

# Fungsi Ekstraksi

In [ ]:
def extract_key_content(text):
    """
    Ekstrak ringkasan fakta, argumen hukum utama, dan amar putusan.
    Disesuaikan dengan struktur khas putusan MA.
    """
    ringkasan_fakta = ""
    argumen_hukum_utama = ""
    amar_putusan = ""

    # Ringkasan Fakta: Cari setelah "Fakta-fakta" atau awalan dakwaan
    fakta_match = re.search(r'(?:Fakta-fakta|Menimbang bahwa Terdakwa telah didakwa oleh Penuntut Umum)\s*(.*?)(?:\s*Menimbang)', text, re.DOTALL | re.IGNORECASE)
    if fakta_match:
        ringkasan_fakta = fakta_match.group(1).strip()
        # Batasi panjang ringkasan fakta
        ringkasan_fakta = ' '.join(ringkasan_fakta.split()[:300]) + '...' if len(ringkasan_fakta.split()) > 300 else ringkasan_fakta

    # Argumen Hukum Utama: Cari di bagian "Menimbang" terkait pertimbangan hakim
    argumen_match = re.search(r'(?:Menimbang, bahwa berdasarkan|Menimbang, bahwa permohonan|Menimbang, bahwa setelah memeriksa)\s*(.*?)(?:\s*MENGADILI|Mengingat)', text, re.DOTALL | re.IGNORECASE)
    if argumen_match:
        argumen_hukum_utama = argumen_match.group(1).strip()
        # Batasi panjang argumen
        argumen_hukum_utama = ' '.join(argumen_hukum_utama.split()[:300]) + '...' if len(argumen_hukum_utama.split()) > 300 else argumen_hukum_utama

    # Amar Putusan: Cari di bawah "MENGADILI:"
    amar_match = re.search(r'MENGADILI:\s*(.*?)(?:Demikianlah|Panitera Pengganti)', text, re.DOTALL | re.IGNORECASE)
    if amar_match:
        amar_putusan = amar_match.group(1).strip()
        # Batasi panjang amar putusan
        amar_putusan = ' '.join(amar_putusan.split()[:200]) + '...' if len(amar_putusan.split()) > 200 else amar_putusan

    return ringkasan_fakta, argumen_hukum_utama, amar_putusan

# Case Representation dan Penyimpanan Data

In [ ]:
def process_cases_for_representation(raw_data_dir, processed_data_dir, output_filename_csv):
    """
    Memproses file .txt, mengekstrak informasi, dan menyimpannya ke CSV.
    """
    case_data = []
    case_id_counter = 1

    # Dapatkan daftar file .txt dari direktori
    txt_files = [f for f in os.listdir(raw_data_dir) if f.endswith('.txt')]

    if not txt_files:
        print(f"ERROR: Tidak ada file .txt ditemukan di '{raw_data_dir}'. Pastikan path sudah benar dan folder berisi file .txt.")
        return

    print(f"Ditemukan {len(txt_files)} file .txt di '{raw_data_dir}' untuk diproses.")

    for filename in txt_files:
        file_path = os.path.join(raw_data_dir, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                full_text = f.read()

            # Ekstraksi Metadata dan Konten Kunci
            metadata = extract_metadata(full_text)
            ringkasan_fakta, argumen_hukum_utama, amar_putusan = extract_key_content(full_text)

            # Feature Engineering: Jumlah kata dalam teks penuh
            text_full_length = len(full_text.split())

            # Simpan semua informasi ke dalam dictionary
            case_info = {
                'case_id': case_id_counter,
                'filename': filename,
                'no_perkara': metadata['no_perkara'],
                'tanggal': metadata['tanggal'],
                'jenis_perkara': metadata['jenis_perkara'],
                'pasal': metadata['pasal'],
                'pihak': metadata['pihak'],
                'ringkasan_fakta': ringkasan_fakta,
                'argumen_hukum_utama': argumen_hukum_utama,
                'amar_putusan': amar_putusan,
                'text_full_length': text_full_length,
                'text_full': full_text # Simpan teks lengkap jika diperlukan untuk retrieval atau embedding
            }
            case_data.append(case_info)
            case_id_counter += 1
            print(f"Processed {filename}")

        except Exception as e:
            print(f"Error processing file {filename}: {e}. Skipping this file.")

    # Buat DataFrame dari data yang terkumpul
    df_cases = pd.DataFrame(case_data)

    # Simpan DataFrame ke CSV
    output_path_csv = os.path.join(processed_data_dir, output_filename_csv)
    df_cases.to_csv(output_path_csv, index=False)

    print(f"\n--- Proses Selesai ---")
    print(f"Data representasi kasus berhasil disimpan ke: {output_path_csv}")
    print(f"Jumlah kasus yang berhasil diproses: {len(df_cases)}")
    print("\nContoh 5 baris pertama data hasil representasi:")
    print(df_cases.head())
    print("\nInformasi DataFrame:")
    df_cases.info()

# Fungsi Utama
if __name__ == "__main__":
    process_cases_for_representation(RAW_DATA_DIR, PROCESSED_DATA_DIR, OUTPUT_FILENAME_CSV)

Ditemukan 51 file .txt di '/content/drive/My Drive/UAS Penalaran Komputer/txt_files' untuk diproses.
Processed 873_Pid.Sus_2024_PN Rap.txt
Processed 946_Pid.Sus_2024_PN Rap.txt
Processed 871_Pid.Sus_2024_PN Rap.txt
Processed 841_Pid.Sus_2024_PN Rap.txt
Processed 849_Pid.Sus_2024_PN Rap.txt
Processed 820_Pid.Sus_2024_PN Rap.txt
Processed 953_Pid.Sus_2024_PN Rap.txt
Processed 861_Pid.Sus_2024_PN Rap.txt
Processed 944_Pid.Sus_2024_PN Rap.txt
Processed 817_Pid.Sus_2024_PN Rap.txt
Processed 842_Pid.Sus_2024_PN Rap.txt
Processed 754_Pid.Sus_2024_PN Rap.txt
Processed 726_Pid.Sus_2024_PN Rap.txt
Processed 917_Pid.Sus_2024_PN Rap.txt
Processed 854_Pid.Sus_2024_PN Rap.txt
Processed 799_Pid.Sus_2024_PN Rap.txt
Processed 890_Pid.Sus_2024_PN Rap.txt
Processed 831_Pid.Sus_2024_PN Rap.txt
Processed 932_Pid.Sus_2024_PN Rap.txt
Processed 748_Pid.Sus_2024_PN Rap.txt
Processed 951_Pid.Sus_2024_PN Rap.txt
Processed 715_Pid.Sus_2024_PN Rap.txt
Processed 729_Pid.Sus_2024_PN Rap.txt
Processed 716_Pid.Sus_202